In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Load CSV file
df = pd.read_csv("EURUSD_tick_OK-2.csv")

# Input–output separation
X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()

# Dataset split
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Normalization
scaler = StandardScaler()
X_tr = scaler.fit_transform(X_tr)
X_te = scaler.transform(X_te)

# Conv1D requires 3D input
X_tr = X_tr.reshape(X_tr.shape[0], X_tr.shape[1], 1)
X_te = X_te.reshape(X_te.shape[0], X_te.shape[1], 1)

# Epoch calculation from SID
sid_code = "2425274"   # update if needed
z_val = int(sid_code[-2])
y_val = int(sid_code[-1])

epochs = 10 if (z_val == 0 and y_val == 0) else (10 + y_val if z_val == 0 else z_val + y_val)

# CNN architecture
cnn_model = Sequential()
cnn_model.add(Conv1D(32, 2, activation="relu", input_shape=(X_tr.shape[1], 1)))
cnn_model.add(MaxPooling1D(2))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation="relu"))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(1))

# Model setup
cnn_model.compile(
    optimizer="adam",
    loss="mean_absolute_error",
    metrics=["mae"]
)

# Training
cnn_model.fit(
    X_tr,
    y_tr,
    epochs=epochs,
    batch_size=50,
    validation_data=(X_te, y_te),
    verbose=1
)

# Evaluation
final_mae = cnn_model.evaluate(X_te, y_te, verbose=0)[1]
print(f"CNN Test MAE: {final_mae:.4f}")

# Model summary
cnn_model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/11
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 45s 2ms/step - loss: 0.1227 - mae: 0.1227 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 2/11
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - loss: 0.1210 - mae: 0.1210 - val_loss: 0.1214 - val_mae: 0.1214
Epoch 3/11
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - loss: 0.1211 - mae: 0.1211 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 4/11
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - loss: 0.1212 - mae: 0.1212 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 5/11
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - loss: 0.1213 - mae: 0.1213 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 6/11
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - loss: 0.1213 - mae: 0.1213 - val_loss: 0.1214 - val_mae: 0.1214
Epoch 7/11
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - loss: 0.1211 - mae: 0.1211 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 8/11
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 0.1211 - mae: 0.1211 - val_loss: 0.1214 - val_mae: 0.1214
Epoch 9/11
17600

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 3, 32)          │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,821 (26.65 KB)

 Trainable params: 2,273 (8.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,548 (17.77 KB)